# PyRAT subject ingestion

## Setup

Using local config file (see [01_pipeline](./01_pipeline.ipynb))

In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject

Connecting cbroz@dss-db.datajoint.io:3306


Manual entry

In [ ]:
# Manual Entry
import datajoint as dj; import getpass
dj.config['database.host'] = '172.26.128.53'        # Put the server name between these apostrophe
dj.config['database.user'] = 'danielmk'             # Put your user name between these apostrophe
dj.config['database.password'] = getpass.getpass(prompt='Database password:')
dj.conn()

## Imports and Root Path Setup

Your `dj.config` file should have a section for your BPod root directory under `custom`: `exp_root_data_dir`. This is a list of one or more paths where the ingestion tool will look for the relative paths it is given.

In [ ]:
import datajoint as dj
from element_interface.utils import find_full_path
from adamacs.pipeline import session, event, trial
from adamacs.ingest.bpod import Bpodfile
from adamacs.paths import get_experiment_root_data_dir

bpod_path = "scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
root_dirs = dj.config["custom"]["exp_root_data_dir"]
bpod_path_full = find_full_path(get_experiment_root_data_dir(),bpod_path)

print(f"Root: {root_dirs}\nFull: {bpod_path_full}")

## Initial check of tables

In [3]:
from adamacs.pipeline import session, event, trial

session.Session.delete()
print('Sessions:', len(session.Session()))
print('Trials  :', len(trial.Trial()))
print('Events  :', len(event.Event()))

Deleting 0 rows from `bonn_session`.`session`
Nothing to delete.
Sessions: 0
Trials  : 0
Events  : 0


## Automated ingestion

The function is designed ask for a confirmation before entered into the schema.

In [4]:
bpod_path = "scan9FB2LN5C_WEZ-8701_StimArenaMaster_20220318_165447.mat"
bpod_object = Bpodfile(bpod_path)
bpod_object.ingest()

BPod items to be inserted:
	Subject : WEZ-8701
	Time    : 2022-03-18 16:59:48
	N Trials: 6
	N Events: 22


Check that insertion worked:

In [7]:
trial.TrialEvent & 'trial_id=1'

session_id,trial_id trial number (1-based indexing),event_type,event_start_time (second) relative to recording start
1,1,at_port,47.8766
1,1,at_target,42.0123
1,1,cue,42.0023
1,1,drinking,49.3201
1,1,in_port,49.1511
1,1,reward,47.9766


We can also interact with bpod objects. For example:

In [9]:
bpod_object.trial(1).attributes

{'port_num': '2', 'error': False, 'timeout': False}

In [10]:
bpod_object.trial(1).events

{'cue': 1.2645,
 'at_target': 1.2745,
 'at_port': 7.1388,
 'reward': 7.2387999999999995,
 'in_port': 8.4133,
 'drinking': 8.5823}